In [22]:
import pandas as pd
import json
import glob
import os

In [2]:
import matplotlib.pyplot as plt

In [4]:
!ls ../sampling_experiment/results_sampling/

dsmpling-run-1.txt dsmpling-run-3.txt dsmpling-run-5.txt
dsmpling-run-2.txt dsmpling-run-4.txt


In [5]:
!cat ../sampling_experiment/results_sampling/dsmpling-run-1.txt

[2022-06-10 11:22:33.975] [memory] [info] allocated bitmap managed memory region (2.00 GB, 32.00 MB block size)
[2022-06-10 11:22:33.976] [aws-lambda] [info] Execution over lambda with 10000MB
[2022-06-10 11:22:33.976] [aws-lambda] [info] Adjusted lambda size to 10000MB
[2022-06-10 11:22:34.148] [aws-lambda] [info] Found AWS Lambda function tuplex-lambda-runner (10000MB)
[2022-06-10 11:22:34.149] [aws-lambda] [info] Using Lambda running on x86_64
[2022-06-10 11:22:34.149] [aws-lambda] [info] Lambda client uses JSON AST serialization format.
[2022-06-10 11:22:34.456] [fileinputoperator] [info] found 36 files (7.86 GB) to process.
[2022-06-10 11:22:34.617] [global] [info] sampled s3://tuplex-public/data/flights_all/flights_on_time_performance_2002_01.csv on 256.00 KB
[2022-06-10 11:22:34.617] [fileinputoperator] [info] Parallel sample fetch done.
[2022-06-10 11:22:34.617] [fileinputoperator] [info] Filling sample cache for csv operator took 0.161046s (1 entries, 0 rows)
[2022-06-10 11:22

13: (i64,i64,i64,i64,i64,str,str,i64,str,str,i64,i64,i64,i64,str,str,str,i64,str,i64,i64,i64,i64,str,str,str,i64,str,i64,i64,null,null,null,null,null,str,null,null,null,null,i64,null,null,null,null,null,str,f64,null,f64,f64,null,null,f64,f64,i64,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null)
col 0: i64 (1179), null (1), 
col 1: i64 (1179), f64 (1), 
col 2: i64 (1179), null (1), 
col 3: i64 (1179), f64 (1), 
col 4: i64 (1179), f64 (1), 
col 5: str (1179), f64 (1), 
col 6: str (1179), f64 (1), 
col 7: i64 (1179), f64 (1), 
col 8: str (1179), f64 (1), 
col 9: str (1179), i64 (1), 
col 10: i64 (1178), null (1), 
col 11: i64 (1178), null (1), 
col 12: i64 (1178), null (1), 
col 13: i64 (1178), null (1), 
col 14: str (1178), null (1), 
col 15: str (1178), null (1), 
col 1

In [46]:
path = '../sampling_experiment/results_sampling/dsmpling-run-1.txt'

all_dfs = pd.DataFrame()
paths = glob.glob('../sampling_experiment/results_sampling/dsmpling-run-*.txt')
for path in paths:
    with open(path) as fp:
        lines = fp.readlines()
        run = int(os.path.basename(path).split('-')[-1].replace('.txt', ''))
        data = list(filter(lambda line: line.startswith("[{'sampling_mode':"), lines))[0]
        data = eval(data)
        df = pd.DataFrame(data)
        df['sampling_time'] = df['metrics'].apply(lambda x: x['sampling_time_s'])
        df['run'] = run
        all_dfs = pd.concat((all_dfs, df))
        
        # find the Lambda Stage result, should be always the same...
        needle = 'LAMBDA Stage:'
        f_lines = list(filter(lambda line: needle in line, lines))
        for L in f_lines:
            L = L[L.find('LAMBDA'):].strip()
            print(L)
        
df = all_dfs

LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exc

In [37]:
df

,sampling_mode,job_time,metrics,sampling_time,run
0,SamplingMode.FIRST_FILE|FIRST_ROWS,31.618398,"{'logical_optimization_time_s': 0.013556, 'llv...",0.781845,4
1,SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS,32.004810,"{'logical_optimization_time_s': 0.0223987, 'll...",1.075170,4
2,SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS,8.514030,"{'logical_optimization_time_s': 0.0192503, 'll...",1.060400,4
3,SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FI...,10.244319,"{'logical_optimization_time_s': 0.0400462, 'll...",1.463310,4
4,SamplingMode.ALL_FILES|FIRST_ROWS,21.220813,"{'logical_optimization_time_s': 0.342812, 'llv...",10.781800,4
5,SamplingMode.ALL_FILES|LAST_ROWS|FIRST_ROWS,51.457322,"{'logical_optimization_time_s': 0.592472, 'llv...",17.664600,4
0,SamplingMode.FIRST_FILE|FIRST_ROWS,31.527039,"{'logical_optimization_time_s': 0.0138061, 'll...",0.755441,5
1,SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS,30.899796,"{'logical_optimization_time_s': 0.0222524, 'll...",1.172620,5
2,SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS,8.351383,"{'logical_optimization_time_s': 0.0184972, 'll...",0.976829,5
3,SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FI...,10.778904,"{'logical_optimization_time_s': 0.0549152, 'll...",1.455600,5


In [53]:
df['job_time_excl_sampling'] = df['job_time'] - df['sampling_time']
gdf = df.groupby('sampling_mode').agg(['mean', 'std']).reset_index().sort_values(by=('job_time', 'mean'))

gdf['sampling_mode'] = gdf['sampling_mode'].apply(lambda x: x.replace('SamplingMode.', '').replace('|', ', '))
gdf

sampling_mode   job_time            \
                                                      mean       std   
4             LAST_FILE, FIRST_FILE, FIRST_ROWS   8.857737  0.551715   
5  LAST_FILE, FIRST_FILE, LAST_ROWS, FIRST_ROWS   9.899837  0.820298   
0                         ALL_FILES, FIRST_ROWS  22.135003  1.767235   
3             FIRST_FILE, LAST_ROWS, FIRST_ROWS  31.704844  0.477530   
2                        FIRST_FILE, FIRST_ROWS  32.292975  1.810578   
1              ALL_FILES, LAST_ROWS, FIRST_ROWS  51.230691  0.539962   

  sampling_time            run           job_time_excl_sampling            
           mean       std mean       std                   mean       std  
4      1.049332  0.160180  3.0  1.581139               7.808405  0.569083  
5      1.493928  0.150532  3.0  1.581139               8.405909  0.751509  
0     11.120540  0.616264  3.0  1.581139              11.014463  1.268094  
3      1.149552  0.043769  3.0  1.581139              30.555292  0.499205  
2      0.810279  0.080171  3.0  1.581139              31.482696  1.771990  
1     17.414840  0.246980  3.0  1.581139              33.815851  0.672447

In [31]:
df

,sampling_mode,job_time,metrics,sampling_time,run
0,SamplingMode.FIRST_FILE|FIRST_ROWS,35.489584,"{'logical_optimization_time_s': 0.0268413, 'll...",0.865639,1
1,SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS,31.815970,"{'logical_optimization_time_s': 0.0218028, 'll...",1.183940,1
2,SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS,9.657309,"{'logical_optimization_time_s': 0.0187468, 'll...",0.892451,1
3,SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FI...,9.056605,"{'logical_optimization_time_s': 0.0395571, 'll...",1.427380,1
4,SamplingMode.ALL_FILES|FIRST_ROWS,20.667818,"{'logical_optimization_time_s': 0.339149, 'llv...",10.811400,1
5,SamplingMode.ALL_FILES|LAST_ROWS|FIRST_ROWS,51.874460,"{'logical_optimization_time_s': 0.587226, 'llv...",17.350500,1
